# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выберите данные по странам своей группы (совместно): <br>
    3530203_80101: Spain, Portugal, France, Italy, Belgium<br>
    3530203_80102: Germany, Netherlands <br>
    3530903_80301: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_80302: Sweden, Finland, Norway, Denmark, Iceland<br>
    
2. Попытайтесь найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Найдите и объясните хотя бы 5 правил.
3. Выведите эти правила в отдельных ячейках. 
4. Подумайте, как можно было бы использовать полученные правила на практике.

In [55]:
import pandas as pd

In [56]:
data = pd.read_csv("lastfm.csv")
data.head(20)

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
5,1,schandmaul,f,Germany
6,1,edguy,f,Germany
7,1,jack johnson,f,Germany
8,1,eluveitie,f,Germany
9,1,the killers,f,Germany


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289955 entries, 0 to 289954
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user     289955 non-null  int64 
 1   artist   289955 non-null  object
 2   sex      289955 non-null  object
 3   country  289955 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.8+ MB


In [58]:
data.describe()

,user
count,289955.000000
mean,9852.460447
std,5692.355041
min,1.000000
25%,4935.000000
50%,9838.000000
75%,14769.000000
max,19718.000000


**Удалим пропущенные значения**

In [59]:

import numpy as np

data.replace(to_replace='[unknown]',value=np.NaN, inplace=True)
data.dropna(how='any', axis=0, inplace=True)
data.isnull().sum()

In [60]:

data_group_by_artist = data.groupby(['user', 'sex', 'country'])['artist'].apply(lambda x: '|'.join(x)).reset_index()
data_group_by_artist

,user,sex,country,artist
0,1,f,Germany,red hot chili peppers|the black dahlia murder|...
1,3,m,United States,devendra banhart|boards of canada|cocorosie|ap...
2,4,m,United Kingdom,tv on the radio|tool|kyuss|dj shadow|air|a tri...
3,5,m,Finland,dream theater|ac/dc|metallica|iron maiden|bob ...
4,6,m,Portugal,lily allen|kanye west|sigur rós|pink floyd|ste...
...,...,...,...,...
14990,19713,m,Ukraine,armin van buuren|above & beyond|atb|ferry cors...
14991,19714,m,United Kingdom,misfits|type o negative|arch enemy|red hot chi...
14992,19715,m,United Kingdom,abba|james blunt|jason mraz|amy winehouse|quee...
14993,19717,m,Brazil,marilyn manson|beyoncé|madonna|t.a.t.u.|katy p...


In [61]:
data_group_by_artist['country'].value_counts()

**Отфильтруем по странам - Spain, Portugal, France, Italy, Belgium**

In [62]:
data_group_by_artist_country = data_group_by_artist[data_group_by_artist.country.isin(['Spain', 'Portugal', 'Italy', 'Belgium', 'France'])]
data_group_by_artist_country

,user,sex,country,artist
4,6,m,Portugal,lily allen|kanye west|sigur rós|pink floyd|ste...
7,12,m,Italy,queen|the beatles|lynyrd skynyrd|neil young|th...
24,36,f,Italy,m.i.a.|fugazi|johnny cash|clint mansell|elton ...
34,47,m,Italy,pj harvey|radiohead|zero 7|einstürzende neubau...
51,74,m,Spain,in extremo|nine inch nails|billy talent|panic!...
...,...,...,...,...
14970,19690,m,Italy,pink floyd|the beatles|the white stripes|depec...
14973,19693,m,Belgium,air|simon & garfunkel|pixies|serge gainsbourg|...
14975,19697,m,Belgium,underworld|pendulum|the prodigy|sufjan stevens...
14981,19704,m,Italy,keane|pinback|coldplay|travis|calexico|doves|d...


# Dummy encoding

In [63]:
dummy_data_by_user = data_group_by_artist_country['artist'].str.get_dummies('|')
dummy_data = pd.concat([data_group_by_artist_country, dummy_data_by_user], axis = 1)
dummy_data

,user,sex,country,artist,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
4,6,m,Portugal,lily allen|kanye west|sigur rós|pink floyd|ste...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,12,m,Italy,queen|the beatles|lynyrd skynyrd|neil young|th...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,36,f,Italy,m.i.a.|fugazi|johnny cash|clint mansell|elton ...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,47,m,Italy,pj harvey|radiohead|zero 7|einstürzende neubau...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
51,74,m,Spain,in extremo|nine inch nails|billy talent|panic!...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14970,19690,m,Italy,pink floyd|the beatles|the white stripes|depec...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14973,19693,m,Belgium,air|simon & garfunkel|pixies|serge gainsbourg|...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14975,19697,m,Belgium,underworld|pendulum|the prodigy|sufjan stevens...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14981,19704,m,Italy,keane|pinback|coldplay|travis|calexico|doves|d...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
filter_country_data= data[data.country.isin(['Spain','Italy','Portugal','France','Belgium'])]
filter_country_data

,user,artist,sex,country
83,6,lily allen,m,Portugal
84,6,kanye west,m,Portugal
85,6,sigur rós,m,Portugal
86,6,pink floyd,m,Portugal
87,6,stevie wonder,m,Portugal
...,...,...,...,...
289758,19707,the decemberists,m,Spain
289759,19707,my bloody valentine,m,Spain
289760,19707,bloc party,m,Spain
289761,19707,new order,m,Spain


In [105]:
pivot_artist = filter_country_data[['user', 'artist']].pivot_table(index='user', columns='artist', aggfunc=any, fill_value=False).astype(int)
pivot_artist.sum()

artist,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,a day to remember,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
user,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
74,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19690,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19693,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Apriori

In [66]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules, fpmax

frequent_itemsets = apriori(dummy_data.drop(columns=['user', 'sex', 'country', 'artist']),
                            min_support=0.02, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.020466,(3 doors down)
1,0.029640,(30 seconds to mars)
2,0.043754,(a perfect circle)
3,0.025406,(abba)
4,0.074100,(ac/dc)
...,...,...
664,0.025406,"(radiohead, muse, the beatles)"
665,0.027523,"(the killers, muse, radiohead)"
666,0.021171,"(the strokes, radiohead, muse)"
667,0.024700,"(the strokes, the killers, muse)"


In [67]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(air),(radiohead),0.092449,0.237826,0.048694,0.526718,2.214714,0.026708,1.610399,1
1,(arcade fire),(radiohead),0.064926,0.237826,0.033874,0.521739,2.193781,0.018433,1.593636,1
2,(kaiser chiefs),(arctic monkeys),0.050812,0.101623,0.025406,0.500000,4.920139,0.020242,1.796754,1
3,(beck),(radiohead),0.047283,0.237826,0.032463,0.686567,2.886842,0.021218,2.431697,1
4,(björk),(radiohead),0.074806,0.237826,0.040932,0.547170,2.300711,0.023141,1.683133,1
...,...,...,...,...,...,...,...,...,...,...
158,"(the strokes, muse)",(radiohead),0.038109,0.237826,0.021171,0.555556,2.335971,0.012108,1.714891,2
159,"(the strokes, the killers)",(muse),0.039520,0.174312,0.024700,0.625000,3.585526,0.017811,2.201835,2
160,"(the strokes, muse)",(the killers),0.038109,0.110092,0.024700,0.648148,5.887346,0.020505,2.529213,2
161,"(the killers, muse)",(the strokes),0.049400,0.087509,0.024700,0.500000,5.713710,0.020377,1.824982,2


In [68]:
rules[ 
       (rules['lift'] > 1) &
       (rules['support'] > 0.04) 
     ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(air),(radiohead),0.092449,0.237826,0.048694,0.526718,2.214714,0.026708,1.610399,1
4,(björk),(radiohead),0.074806,0.237826,0.040932,0.547170,2.300711,0.023141,1.683133,1
10,(moby),(coldplay),0.081157,0.188426,0.040932,0.504348,2.676632,0.025639,1.637386,1
11,(oasis),(coldplay),0.083275,0.188426,0.047989,0.576271,3.058338,0.032298,1.915314,1
14,(the killers),(coldplay),0.110092,0.188426,0.055752,0.506410,2.687578,0.035007,1.644227,1
15,(u2),(coldplay),0.088215,0.188426,0.045166,0.512000,2.717243,0.028544,1.663061,1
23,(interpol),(muse),0.079746,0.174312,0.040932,0.513274,2.944574,0.027031,1.696414,1
24,(interpol),(radiohead),0.079746,0.237826,0.045872,0.575221,2.418660,0.026906,1.794284,1
30,(massive attack),(radiohead),0.088215,0.237826,0.045166,0.512000,2.152831,0.024186,1.561831,1
35,(sigur rós),(radiohead),0.086803,0.237826,0.047989,0.552846,2.324576,0.027345,1.704497,1


**Apriori by country**

In [111]:
pivot_country = filter_country_data[['user','country']].pivot_table(index='user', columns = 'country', aggfunc=any, fill_value=False).astype(int)
pivot_data = pd.concat([pivot_artist,pivot_country], axis=1)
apriori_data = apriori(pivot_data, min_support=0.02, use_colnames=True)
rules_data = association_rules(apriori_data, metric='confidence', min_threshold=0.6)
rules_data.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(keane),(coldplay),0.038109,0.188426,0.033169,0.870370,4.619157,0.025988,6.260712
42,"(interpol, franz ferdinand)",(radiohead),0.029640,0.237826,0.022583,0.761905,3.203617,0.015534,3.201129
36,"(the killers, oasis)",(coldplay),0.032463,0.188426,0.024700,0.760870,4.038023,0.018583,3.393854
14,"(the killers, bloc party)",(coldplay),0.035286,0.188426,0.026112,0.740000,3.927266,0.019463,3.121437
30,"(muse, oasis)",(coldplay),0.035992,0.188426,0.026112,0.725490,3.850261,0.019330,2.956447
...,...,...,...,...,...,...,...,...,...
7,"(muse, bloc party)",(arctic monkeys),0.037403,0.101623,0.022583,0.603774,5.941300,0.018782,2.267332
45,"(muse, franz ferdinand)",(radiohead),0.037403,0.237826,0.022583,0.603774,2.538716,0.013688,1.923581
23,"(muse, franz ferdinand)",(coldplay),0.037403,0.188426,0.022583,0.603774,3.204297,0.015535,2.048258
59,"(muse, placebo)",(radiohead),0.044460,0.237826,0.026817,0.603175,2.536197,0.016243,1.920677


# FP-growth

In [42]:
fpgrowth_artist = fpgrowth(pivot_artist, min_support=0.005, use_colnames=True)
len(fpgrowth_artist)

13871

In [43]:
fpgrowth_artist_rules = association_rules(fpgrowth_artist, metric="confidence", min_threshold=0.8)
fpgrowth_artist_rules.sort_values('support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5024,(keane),(coldplay),0.038109,0.188426,0.033169,0.870370,4.619157,0.025988,6.260712
389,"(the killers, muse, bloc party)",(coldplay),0.023289,0.188426,0.019760,0.848485,4.503008,0.015372,5.356387
851,"(interpol, arcade fire)",(the strokes),0.022583,0.087509,0.018349,0.812500,9.284778,0.016372,4.866620
2593,"(the killers, muse, oasis)",(coldplay),0.020466,0.188426,0.016937,0.827586,4.392096,0.013081,4.707128
942,"(the killers, death cab for cutie)",(coldplay),0.021171,0.188426,0.016937,0.800000,4.245693,0.012948,4.057869
...,...,...,...,...,...,...,...,...,...
2619,"(the strokes, oasis, placebo, arctic monkeys)",(coldplay),0.006351,0.188426,0.006351,1.000000,5.307116,0.005155,inf
2621,"(the killers, oasis, nirvana)",(coldplay),0.007763,0.188426,0.006351,0.818182,4.342186,0.004889,4.463656
2622,"(the strokes, nirvana, oasis)",(coldplay),0.006351,0.188426,0.006351,1.000000,5.307116,0.005155,inf
2624,"(the strokes, radiohead, oasis, the beatles)",(coldplay),0.007763,0.188426,0.006351,0.818182,4.342186,0.004889,4.463656


In [44]:
fpgrowth_artist_rules['lift'].describe()

In [54]:
fpgrowth_artist_rules_lift = fpgrowth_artist_rules[fpgrowth_artist_rules['lift'] > 15]
fpgrowth_artist_rules_lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
33,"(radiohead, snow patrol, the kooks)","(the killers, muse)",0.007763,0.049400,0.007057,0.909091,18.402597,0.006674,10.456598
65,"(snow patrol, oasis, bloc party)","(the killers, coldplay)",0.007763,0.055752,0.007057,0.909091,16.306099,0.006624,10.386733
69,"(snow patrol, oasis, bloc party)","(the killers, muse)",0.007763,0.049400,0.006351,0.818182,16.562338,0.005968,5.228299
75,"(muse, oasis, snow patrol, bloc party)","(the killers, coldplay)",0.007057,0.055752,0.006351,0.900000,16.143038,0.005958,9.442484
76,"(the killers, muse, oasis, snow patrol)","(coldplay, bloc party)",0.007763,0.038814,0.006351,0.818182,21.079339,0.006050,5.286521
...,...,...,...,...,...,...,...,...,...
5485,"(the smashing pumpkins, pearl jam, metallica)",(deftones),0.007057,0.035286,0.006351,0.900000,25.506000,0.006102,9.647142
5486,"(kreator, judas priest)",(megadeth),0.008469,0.038814,0.007057,0.833333,21.469697,0.006728,5.767114
5487,"(kreator, metallica)",(megadeth),0.007763,0.038814,0.006351,0.818182,21.079339,0.006050,5.286521
5488,"(kreator, iron maiden)",(megadeth),0.007057,0.038814,0.006351,0.900000,23.187273,0.006078,9.611856


# FP-growth by country

In [120]:
fpgrowth_data = fpgrowth(pivot_data, min_support=0.007, use_colnames=True)
len(fpgrowth_data)

12287

In [121]:
fpgrowth_country_rules = association_rules(fpgrowth_data, metric="confidence", min_threshold=0.8)
fpgrowth_country_rules.sort_values('support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2990,(keane),(coldplay),0.038109,0.188426,0.033169,0.870370,4.619157,0.025988,6.260712
278,"(the killers, muse, bloc party)",(coldplay),0.023289,0.188426,0.019760,0.848485,4.503008,0.015372,5.356387
433,"(interpol, arcade fire)",(the strokes),0.022583,0.087509,0.018349,0.812500,9.284778,0.016372,4.866620
481,"(the killers, death cab for cutie)",(coldplay),0.021171,0.188426,0.016937,0.800000,4.245693,0.012948,4.057869
1484,"(the killers, muse, oasis)",(coldplay),0.020466,0.188426,0.016937,0.827586,4.392096,0.013081,4.707128
...,...,...,...,...,...,...,...,...,...
1429,"(oasis, the hives, arctic monkeys)","(the killers, coldplay)",0.007763,0.055752,0.007057,0.909091,16.306099,0.006624,10.386733
1428,"(coldplay, oasis, the hives)","(the killers, arctic monkeys)",0.007057,0.037403,0.007057,1.000000,26.735849,0.006793,inf
1427,"(the killers, coldplay, the hives, arctic monk...",(oasis),0.008469,0.083275,0.007057,0.833333,10.007062,0.006352,5.500353
1426,"(the killers, oasis, the hives, arctic monkeys)",(coldplay),0.007763,0.188426,0.007057,0.909091,4.824651,0.005594,8.927311


In [122]:
fpgrowth_country_rules_lift = fpgrowth_country_rules[fpgrowth_country_rules['lift'] > 15]
fpgrowth_country_rules_lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
24,"(radiohead, snow patrol, the kooks)","(the killers, muse)",0.007763,0.049400,0.007057,0.909091,18.402597,0.006674,10.456598
50,"(snow patrol, oasis, bloc party)","(the killers, coldplay)",0.007763,0.055752,0.007057,0.909091,16.306099,0.006624,10.386733
93,"(the strokes, snow patrol, interpol)","(the killers, coldplay)",0.007057,0.055752,0.007057,1.000000,17.936709,0.006664,inf
130,"(the rolling stones, led zeppelin, the beatles...",(the who),0.008469,0.044460,0.007057,0.833333,18.743386,0.006681,5.733239
356,"(the strokes, franz ferdinand, Spain, bloc party)","(the killers, coldplay)",0.009174,0.055752,0.007763,0.846154,15.177215,0.007251,6.137615
...,...,...,...,...,...,...,...,...,...
3208,"(leona lewis, Spain, rihanna)",(beyoncé),0.008469,0.032463,0.007057,0.833333,25.670290,0.006782,5.805222
3209,"(leona lewis, Spain, beyoncé)",(rihanna),0.007763,0.040226,0.007057,0.909091,22.599681,0.006745,10.557516
3211,"(leona lewis, nelly furtado)",(rihanna),0.007763,0.040226,0.007057,0.909091,22.599681,0.006745,10.557516
3212,"(leona lewis, nelly furtado)",(beyoncé),0.007763,0.032463,0.007057,0.909091,28.003953,0.006805,10.642908


# FPmax

In [69]:
fpmax_artist = fpmax(pivot_artist, min_support=0.05, use_colnames=True)
len(fpmax_artist)

72

In [89]:
fpmax_artist_rules = association_rules(fpmax_artist, metric='support', min_threshold=0.05, support_only=True)
fpmax_artist_rules.sort_values('support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
12,(radiohead),(coldplay),NaN,NaN,0.083275,NaN,NaN,NaN,NaN
13,(coldplay),(radiohead),NaN,NaN,0.083275,NaN,NaN,NaN,NaN
10,(radiohead),(muse),NaN,NaN,0.080452,NaN,NaN,NaN,NaN
11,(muse),(radiohead),NaN,NaN,0.080452,NaN,NaN,NaN,NaN
6,(radiohead),(the beatles),NaN,NaN,0.070572,NaN,NaN,NaN,NaN
7,(the beatles),(radiohead),NaN,NaN,0.070572,NaN,NaN,NaN,NaN
8,(muse),(coldplay),NaN,NaN,0.068454,NaN,NaN,NaN,NaN
9,(coldplay),(muse),NaN,NaN,0.068454,NaN,NaN,NaN,NaN
0,(the killers),(coldplay),NaN,NaN,0.055752,NaN,NaN,NaN,NaN
1,(coldplay),(the killers),NaN,NaN,0.055752,NaN,NaN,NaN,NaN
